# 캘리포니아 집값 데이터 분석 2020

이 노트북은 캘리포니아 집값 데이터(`data/california-house-prices/train.csv`)를 분석하여 특징을 파악하고, 머신러닝 모델링을 위한 전처리 전략을 수립하기 위해 작성되었습니다.

## 분석 목표
1. 데이터 구조 및 기초 통계 확인
2. 타겟 변수(`Sold Price`)와 데이처 특징 간의 상관관계 분석
3. 결측치(Missing Values) 확인 및 처리 방안 제시
4. 주요 피처(Feature) 분석 및 가공 아이디어 도출

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 시각화 설정
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.family'] = 'Malgun Gothic' # 한글 폰트 설정 (Windows)
plt.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings('ignore')

## 1. 데이터 로드

In [ ]:
file_path = 'data/california-house-prices/train.csv'
df = pd.read_csv(file_path)
print(f"데이터 크기: {df.shape}")
df.head()

## 2. 데이터 기본 정보 확인
데이터의 컬럼 타입, 결측치 개수, 기초 통계량을 확인합니다.

In [ ]:
df.info()

In [ ]:
df.describe()

## 3. 타겟 변수(Sold Price) 분석
우리가 예측해야 할 `Sold Price`의 분포를 확인합니다. 가격 데이터는 왜도(Skewness)가 높은 경우가 많으므로 로그 변환 필요성을 검토합니다.

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df['Sold Price'], kde=True, bins=50)
plt.title('Distribution of Sold Price')
plt.xlabel('Sold Price')
plt.show()

In [ ]:
print(f"왜도(Skewness): {df['Sold Price'].skew()}")
print(f"첨도(Kurtosis): {df['Sold Price'].kurt()}")

**분석 코멘트:**
- `Sold Price`의 분포가 왼쪽으로 치우쳐져 있다면(왜도가 높다면), `np.log1p()`를 사용하여 로그 변환을 적용하는 것이 학습에 도움이 될 수 있습니다.

## 4. 결측치(Missing Value) 분석
각 컬럼별 결측치 비율을 확인하고 처리 전략을 세웁니다.

In [ ]:
missing_values = df.isnull().sum().sort_values(ascending=False)
missing_percent = (df.isnull().sum() / len(df)) * 100
missing_info = pd.concat([missing_values, missing_percent], axis=1, keys=['Total', 'Percent'])
missing_info = missing_info[missing_info['Total'] > 0]

plt.figure(figsize=(12, 8))
sns.barplot(x=missing_info.index, y=missing_info['Percent'])
plt.xticks(rotation=90)
plt.title('Percent of Missing Values by Feature')
plt.ylabel('Percent (%)')
plt.show()

display(missing_info)

## 5. 상관관계 분석
수치형 변수들과 타겟 변수 간의 상관관계를 확인합니다.

In [ ]:
# 수치형 컬럼만 선택
numeric_df = df.select_dtypes(include=[np.number])
corr_matrix = numeric_df.corr()

plt.figure(figsize=(14, 12))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', square=True)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
# Sold Price와 상관관계가 높은 상위 10개 피처 확인
k = 10
cols = corr_matrix.nlargest(k, 'Sold Price')['Sold Price'].index
print("Sold Price와 가장 상관관계가 높은 10개 피처:")
print(cols.values)

## 6. 주요 범주형 변수 분석 및 가공 아이디어

### City / Region
지역 정보는 집값에 매우 큰 영향을 미칩니다. `City`나 `Zip` 코드를 활용하여 지역별 평균 가격을 피처로 추가하거나, 상위 빈도 지역만 남기고 나머지는 'Other'로 묶는 방식을 고려할 수 있습니다.

In [ ]:
top_cities = df['City'].value_counts().head(20)
print("Top 20 Cities by count:")
print(top_cities)

plt.figure(figsize=(12, 6))
sns.boxplot(x='City', y='Sold Price', data=df[df['City'].isin(top_cities.index)])
plt.xticks(rotation=90)
plt.title('Sold Price Distribution by Top 20 Cities')
plt.show()

### Type (주거 형태)
주거 형태(`SingleFamily`, `Condo` 등)에 따른 가격 차이를 봅니다.

In [ ]:
plt.figure(figsize=(12, 6))
top_types = df['Type'].value_counts().head(10).index
sns.boxplot(x='Type', y='Sold Price', data=df[df['Type'].isin(top_types)])
plt.xticks(rotation=45)
plt.title('Sold Price by Type')
plt.show()

## 7. 날짜 데이터 전처리 아이디어
`Listed On`, `Last Sold On` 등의 날짜 데이터를 활용하여 연도, 월, 판매까지 걸린 시간 등을 추출할 수 있습니다.

In [ ]:
# 날짜 타입 변환 예시
try:
    df['Listed On'] = pd.to_datetime(df['Listed On'])
    df['Listed Year'] = df['Listed On'].dt.year
    df['Listed Month'] = df['Listed On'].dt.month
    
    plt.figure(figsize=(10,6))
    sns.boxplot(x='Listed Year', y='Sold Price', data=df)
    plt.title('Sold Price by Listed Year')
    plt.show()
except Exception as e:
    print("날짜 변환 중 오류 발생 (또는 이미 변환됨):", e)

## 8. 전처리 및 피처 엔지니어링 제안 요약

1. **결측치 처리**: 
    - 결측 비율이 매우 높은 컬럼(예: 50% 이상)은 삭제 고려.
    - 수치형 데이터는 중위값(median) 또는 평균(mean) 대치.
    - 범주형 데이터는 최빈값(mode) 또는 'Unknown'으로 대치.
    - `Lot`, `Garagespaces` 등이 0 또는 결측인 경우, 실제 없는 것인지 누락인지 판단 필요 (보통 0으로 채움).

2. **이상치(Outlier) 제거**:
    - `Sold Price`나 `Total interior livable area` 등 주요 수치형 변수에서 IQR 기반 또는 산점도 시각화를 통해 극단적인 이상치를 제거.

3. **범주형 변수 인코딩**:
    - `Type`, `City`, `Heating`, `Cooling` 등은 One-Hot Encoding 또는 Label Encoding 적용.
    - 카테고리가 너무 많은 경우(`City` 등) 빈도수 기준 상위 n개만 남기고 나머지는 'Other' 처리.

4. **파생 변수 생성**:
    - `Year built` -> `House Age` (현재 연도 - 건축 연도)
    - `Elementary School Score` 등 학교 점수 활용
    - `Bedrooms`, `Bathrooms` 합쳐서 `Total Rooms` 생성 고려

이 노트북을 바탕으로 데이터 전처리를 수행하고 모델링을 진행하시면 됩니다.